# Movies Database

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_table(r"datasets/movie datasets/imdb/title.basics.tsv/data.tsv", sep="\t")

C:\Users\smend\AppData\Local\Temp\ipykernel_16004\1166172987.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(r"datasets/movie datasets/imdb/title.basics.tsv/data.tsv", sep="\t")


In [ ]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# Text 2 SQL

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from pprint import pprint

In [ ]:
dataset = load_dataset("b-mc2/sql-create-context", )
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'context', 'question'],
        num_rows: 78577
    })
})

In [ ]:
def generate_training_prompt(question: str, answer: str, context: str ) -> str:
    return {"text": f"""
You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{question.strip()}

### Context:
{context.strip()}

### Response:
{answer.strip()}
  """.strip()}

In [ ]:
example = dataset["train"][5]
prompt = generate_training_prompt(example["question"], example["answer"], example["context"])
print(prompt["text"])

You are a powerful text-to-SQL model. Your job is to answer questions about a database. You are given a question and context regarding one or more tables. 

You must output the SQL query that answers the question.

### Input:
What are the names of the heads who are born outside the California state?

### Context:
CREATE TABLE head (name VARCHAR, born_state VARCHAR)

### Response:
SELECT name FROM head WHERE born_state <> 'California'


In [ ]:
def process_dataset(data: Dataset):
  return(
      data.shuffle(seed=42)
      .map(lambda data_point: generate_training_prompt(data_point["question"], data_point["answer"], data_point["context"]))
      )

In [ ]:
for data in dataset["train"].select([0, 25, 58, 222, 567]):
    print(data)

In [ ]:
processed_dataset = process_dataset(dataset["train"])

In [ ]:
final_dataset = DatasetDict({"train": Dataset.from_dict(processed_dataset[:60_000]),
            "validation": Dataset.from_dict(processed_dataset[60_000:70_000]),
            "test": Dataset.from_dict(processed_dataset[70_000:])})

NameError: name 'processed_dataset' is not defined

In [ ]:
final_dataset.save_to_disk("datasets/complete.hf")

Saving the dataset (0/1 shards):   0%|          | 0/60000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8577 [00:00<?, ? examples/s]

# Loading Dataset

In [ ]:
from datasets import Dataset, load_dataset

In [ ]:
dataset = load_dataset("datasets/text2sql/10k/", data_dir="datasets/text2sql/10k/train/")
dataset

EmptyDatasetError: The directory at C:/Users/smend/OneDrive/Desktop/Stevens/SEM1/Deep Learning - CS 583 A/Project/datasets/text2sql/10k/datasets/text2sql/10k/train doesn't contain any data files